2D Hands Image 기반 시퀀스 분류 (MobileNetV3 + GRU + Attention)

In [1]:
%%writefile train_seq_cnn_from_hands_mobilenet_v2.py
"""
2D Hands Image 기반 시퀀스 분류 (MobileNetV3 + GRU + Attention)

1) /Holistic_hands_frames/1~10 폴더의 PNG를 직접 읽어서 시퀀스 구성
   - 파일 예시: WORD1496_..._REAL01_U_s00.png ~ s15.png
   - base_id = '..._REAL01_U' 단위로 한 시퀀스로 그룹
2) 각 base_id 를 하나의 시퀀스(sample)로 사용 (길이 T=SEQ_LEN)
3) 라벨은 파일명 내 WORDxxxx 에서 추출 (xxxx 문자열)
4) 전체 시퀀스를 라벨 기준 stratified 0.7 / 0.15 / 0.15 split
5) MobileNetV3 Small 프레임 인코더 + GRU + Attention으로 분류 학습
   - Input: (B, T, 3, H, W)  (2D 이미지를 그대로 사용)
   - 프레임 인코더: MobileNetV3 Small (ImageNet pretrained) → 256차원
   - 시퀀스 인코더: 양방향 GRU (hidden=192, num_layers=2, dropout=0.2)
   - Attention Pooling
   - Class weight 적용
   - Label smoothing
   - 프레임 레벨 Gaussian noise + Temporal dropout (train 시에만)
   - ReduceLROnPlateau 스케줄러 + EarlyStopping
"""

import os
import glob
import re
import random
from typing import List, Dict, Any, Tuple

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

import torchvision.transforms as transforms
import torchvision.models as models

print("[INFO] train_seq_cnn_from_hands_mobilenet_v2.py loaded")

# -------------------------------------------------------------
# 설정
# -------------------------------------------------------------
ROOT = "/content/drive/MyDrive/cv-medislr/data/preprocessed/holistic_frames/hands"

SEQ_LEN = 16       # 시퀀스 길이 (s00~s15 기준)
IMG_SIZE = 96      # 입력 이미지 크기 (HxW)
BATCH = 32         # Colab CPU가 부담되면 16으로 줄여도 됨
EPOCHS = 40
LR = 1e-3
WEIGHT_DECAY = 1e-4
ES_PATIENCE = 7    # Early stopping patience

# 프레임 레벨 데이터 증강
IMG_NOISE_STD = 0.01     # 0이면 비활성화
TEMP_DROP_PROB = 0.10    # 0이면 비활성화 (프레임 드롭)

RANDOM_SEED = 42

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Device = {DEVICE}")

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if DEVICE.type == "cuda":
    torch.cuda.manual_seed_all(RANDOM_SEED)

# -------------------------------------------------------------
# 1. 시퀀스 메타 수집 (2D PNG → 시퀀스 경로 리스트)
# -------------------------------------------------------------

# 예:  WORD1496_..._REAL01_U_s15.png  → base_id='WORD1496_..._REAL01_U', seg_idx=15
SEQ_GROUP_RE = re.compile(r"(.+)_s(\d+)\.png$")


def collect_sequences_from_png(root_dir: str) -> Tuple[List[List[str]], List[str]]:
    """
    각 person 폴더(1~10) 아래 PNG 파일을 읽어서
    base_id 단위 시퀀스를 구성하고, 라벨 문자열을 추출합니다.

    Returns
    -------
    sequences : List[List[str]]
        각 원소는 한 시퀀스를 구성하는 frame path 리스트 (길이 가변)
    labels_str : List[str]
        각 시퀀스에 대응되는 라벨 문자열 (예: "0029")
    """
    sequences: List[List[str]] = []
    labels_str: List[str] = []

    # 1,2,...,10 폴더만 사용 (seq_embeddings_hands 등은 제외)
    person_dirs = sorted(
        [
            d for d in glob.glob(os.path.join(root_dir, "*"))
            if os.path.isdir(d) and os.path.basename(d).isdigit()
        ]
    )

    for p_dir in person_dirs:
        person_id = os.path.basename(p_dir)
        png_files = sorted(glob.glob(os.path.join(p_dir, "*.png")))
        print(f"[SCAN] Person {person_id}: {len(png_files)} frames")

        # base_id 기준 그룹핑
        groups: Dict[str, Dict[int, str]] = {}

        for img_path in png_files:
            fname = os.path.basename(img_path)
            m = SEQ_GROUP_RE.match(fname)
            if not m:
                continue
            base_id, seg_str = m.groups()
            seg_idx = int(seg_str)

            if base_id not in groups:
                groups[base_id] = {}
            # 같은 seg_idx에 여러 파일이 있을 가능성은 거의 없지만, 있다면 가장 앞의 것만 사용
            if seg_idx not in groups[base_id]:
                groups[base_id][seg_idx] = img_path

        # base_id 하나가 하나의 시퀀스가 됨
        for base_id, seg_dict in groups.items():
            seg_indices = sorted(seg_dict.keys())
            frame_paths = [seg_dict[i] for i in seg_indices]

            if len(frame_paths) == 0:
                continue

            # 라벨 문자열 추출: base_id 내 WORDxxxx 사용
            label_txt = "0000"
            try:
                parts = base_id.split("_")
                word_token = next(
                    (p for p in parts if p.startswith("WORD") and p[4:].isdigit()),
                    None,
                )
                if word_token is not None:
                    label_txt = word_token.replace("WORD", "")
            except Exception:
                pass

            sequences.append(frame_paths)
            labels_str.append(label_txt)

    print(f"[META] Total sequence samples = {len(sequences)}")
    return sequences, labels_str


# -------------------------------------------------------------
# 2. Dataset 정의
# -------------------------------------------------------------
class HandsSeqDataset(Dataset):
    """
    sequences: List[List[str]]  (각 원소는 frame path 리스트)
    labels   : List[int]        (encoded label)
    indices  : Subset 인덱스
    """

    def __init__(
        self,
        sequences: List[List[str]],
        labels: List[int],
        indices: List[int],
        transform: transforms.Compose,
    ):
        self.sequences = sequences
        self.labels = labels
        self.indices = indices
        self.transform = transform

    def __len__(self) -> int:
        return len(self.indices)

    def _select_frames(self, frame_paths: List[str]) -> List[str]:
        """
        길이가 가변인 frame_paths를 SEQ_LEN 길이로 변환.
        - 길이가 SEQ_LEN보다 길면 균등 간격으로 샘플링
        - 짧으면 마지막 프레임을 반복하여 패딩
        """
        L = len(frame_paths)
        if L == 0:
            raise ValueError("Empty frame list encountered.")

        if L == SEQ_LEN:
            return frame_paths
        elif L > SEQ_LEN:
            idxs = np.linspace(0, L - 1, SEQ_LEN)
            idxs = np.round(idxs).astype(int)
            idxs = np.clip(idxs, 0, L - 1)
            return [frame_paths[i] for i in idxs]
        else:
            pad_num = SEQ_LEN - L
            return frame_paths + [frame_paths[-1]] * pad_num

    def __getitem__(self, idx: int):
        real_idx = self.indices[idx]
        frame_paths = self.sequences[real_idx]
        frame_paths = self._select_frames(frame_paths)

        imgs = []
        for path in frame_paths:
            img = Image.open(path).convert("RGB")
            img = self.transform(img)  # (3, H, W)
            imgs.append(img)
        # (T, 3, H, W)
        seq_tensor = torch.stack(imgs, dim=0)
        label = self.labels[real_idx]
        return seq_tensor, label


# -------------------------------------------------------------
# 3. MobileNetV3 + GRU + Attention 모델
# -------------------------------------------------------------
class FrameEncoderMobileNetV3(nn.Module):
    """
    ImageNet pretrained MobileNetV3 Small을 프레임 인코더로 사용.
    입력: (B, 3, H, W) → 출력: (B, out_dim)
    """

    def __init__(self, out_dim: int = 256, pretrained: bool = True):
        super().__init__()
        weights = models.MobileNet_V3_Small_Weights.DEFAULT if pretrained else None
        backbone = models.mobilenet_v3_small(weights=weights)

        # feature extractor (classifier 제거)
        self.features = backbone.features
        self.gap = nn.AdaptiveAvgPool2d((1, 1))

        in_feat = backbone.classifier[0].in_features
        self.proj = nn.Linear(in_feat, out_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, 3, H, W)
        f = self.features(x)              # (B, C, H', W')
        f = self.gap(f).flatten(1)        # (B, C)
        f = self.proj(f)                  # (B, out_dim)
        return f


class SeqCNN_MobileNet_GRU_Attn(nn.Module):
    """
    2D 이미지 → MobileNetV3 Small → GRU + Attention → 분류
    GRU/Attention/Head 구조는 CNN 임베딩 GRU 모델과 최대한 동일하게 구성.
    """

    def __init__(
        self,
        num_classes: int,
        frame_out_dim: int = 256,
        hidden_dim: int = 192,
        num_layers: int = 2,
        bidirectional: bool = True,
        dropout: float = 0.2,
        pretrained_backbone: bool = True,
    ):
        super().__init__()
        self.encoder = FrameEncoderMobileNetV3(
            out_dim=frame_out_dim,
            pretrained=pretrained_backbone,
        )

        self.gru = nn.GRU(
            input_size=frame_out_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        out_dim = hidden_dim * (2 if bidirectional else 1)

        self.attn_fc = nn.Linear(out_dim, 1)
        self.head = nn.Sequential(
            nn.LayerNorm(out_dim),
            nn.Dropout(0.1),
            nn.Linear(out_dim, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, T, 3, H, W)
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)

        # 1) 프레임별 CNN 인코딩
        feat = self.encoder(x)               # (B*T, F)
        feat = feat.view(B, T, -1)           # (B, T, F)

        # 2) GRU + Attention
        out, _ = self.gru(feat)              # (B, T, H*)
        out = torch.nan_to_num(out, nan=0.0, posinf=0.0, neginf=0.0)

        w = self.attn_fc(out)                # (B, T, 1)
        w = torch.softmax(w, dim=1)          # (B, T, 1)
        w = torch.nan_to_num(w, nan=0.0, posinf=0.0, neginf=0.0)

        feat_seq = (w * out).sum(dim=1)      # (B, H*)
        logits = self.head(feat_seq)         # (B, num_classes)
        return logits


# -------------------------------------------------------------
# 4. Train / Eval loop  (CNN 임베딩 GRU 버전과 동일 패턴)
# -------------------------------------------------------------
def run_epoch(
    model: nn.Module,
    loader: DataLoader,
    criterion,
    optimizer=None,
    device="cpu",
    phase: str = "train",
):
    train_flag = optimizer is not None
    model.train(train_flag)

    total, correct, loss_sum = 0, 0, 0.0

    for xb, yb in loader:
        # xb: (B, T, 3, H, W)
        xb = xb.to(device)
        yb = yb.to(device)

        # ----- frame-level augmentation (train 시에만) -----
        if train_flag:
            if IMG_NOISE_STD > 0:
                noise = torch.randn_like(xb) * IMG_NOISE_STD
                xb = xb + noise

            if TEMP_DROP_PROB > 0:
                # 일부 프레임 전체를 drop (0으로)
                B, T, C, H, W = xb.shape
                mask = (
                    torch.rand(B, T, 1, 1, 1, device=device)
                    > TEMP_DROP_PROB
                ).float()
                xb = xb * mask

        if train_flag:
            optimizer.zero_grad(set_to_none=True)

        logits = model(xb)
        loss = criterion(logits, yb)

        if train_flag:
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 3.0)
            optimizer.step()

        with torch.no_grad():
            pred = logits.argmax(dim=1)
            total += yb.size(0)
            correct += (pred == yb).sum().item()
            loss_sum += loss.item() * yb.size(0)

    avg_loss = loss_sum / max(1, total)
    acc = correct / max(1, total)
    return avg_loss, acc


def eval_with_confusion(model, loader, device="cpu"):
    model.eval()
    all_y, all_p = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)
            logits = model(xb)
            pred = logits.argmax(dim=1)
            all_y += yb.cpu().tolist()
            all_p += pred.cpu().tolist()
    cm = confusion_matrix(all_y, all_p)
    acc = (cm.diagonal().sum() / cm.sum()) if cm.sum() > 0 else 0.0
    return acc, cm


# -------------------------------------------------------------
# 5. main
# -------------------------------------------------------------
if __name__ == "__main__":
    # 1) 시퀀스 메타 수집
    seq_paths, labels_str = collect_sequences_from_png(ROOT)

    if len(seq_paths) == 0:
        raise ValueError("수집된 시퀀스가 없습니다. 폴더 구조와 파일명을 확인하세요.")

    # 라벨 인코딩 (문자열 → 정수)
    encoder = LabelEncoder()
    encoder.fit(labels_str)
    labels = encoder.transform(labels_str).astype(np.int64)
    num_classes = len(encoder.classes_)
    print(f"[META] num_classes = {num_classes}, labels = {encoder.classes_}")

    # 2) Stratified random split
    y_all = labels
    indices = np.arange(len(seq_paths))

    train_idx, temp_idx = train_test_split(
        indices,
        test_size=0.3,
        stratify=y_all,
        random_state=RANDOM_SEED,
    )
    val_idx, test_idx = train_test_split(
        temp_idx,
        test_size=0.5,
        stratify=y_all[temp_idx],
        random_state=RANDOM_SEED,
    )

    print(f"[SPLIT] train={len(train_idx)}, val={len(val_idx)}, test={len(test_idx)}")

    # 3) class weight 계산 (임베딩 GRU와 동일 방식)
    class_counts = np.bincount(y_all[train_idx], minlength=num_classes)
    max_count = class_counts.max()
    class_weights = (max_count / class_counts).astype(np.float32)
    print("[INFO] class weights:", class_weights)

    # 4) Transform 정의 (ImageNet 정규화 사용)
    imagenet_mean = [0.485, 0.456, 0.406]
    imagenet_std = [0.229, 0.224, 0.225]

    transform_train = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])

    transform_eval = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])

    # 5) Dataset / DataLoader
    train_set = HandsSeqDataset(seq_paths, y_all.tolist(), train_idx.tolist(), transform_train)
    val_set   = HandsSeqDataset(seq_paths, y_all.tolist(), val_idx.tolist(), transform_eval)
    test_set  = HandsSeqDataset(seq_paths, y_all.tolist(), test_idx.tolist(), transform_eval)

    train_dl = DataLoader(train_set, batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=(DEVICE.type=="cuda"))
    val_dl   = DataLoader(val_set, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=(DEVICE.type=="cuda"))
    test_dl  = DataLoader(test_set, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=(DEVICE.type=="cuda"))

    # 6) 모델 정의
    model = SeqCNN_MobileNet_GRU_Attn(
        num_classes=num_classes,
        frame_out_dim=256,
        hidden_dim=192,
        num_layers=2,
        bidirectional=True,
        dropout=0.2,
        pretrained_backbone=True,
    ).to(DEVICE)

    # 필요하면 backbone freeze (RAM/속도 절약용)
    # for p in model.encoder.features.parameters():
    #     p.requires_grad = False

    weight_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)
    criterion = nn.CrossEntropyLoss(weight=weight_tensor, label_smoothing=0.05)

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LR,
        weight_decay=WEIGHT_DECAY,
    )

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode="max",
        factor=0.5,
        patience=3,
        min_lr=1e-5,
    )

    best_val_acc = 0.0
    best_state = None
    epochs_no_improve = 0

    save_path = os.path.join(ROOT, "seq_cnn_mobilenet_gru_attn_hands_best.pt")

    # 7) 학습 루프
    for epoch in range(1, EPOCHS + 1):
        tr_loss, tr_acc = run_epoch(model, train_dl, criterion, optimizer, DEVICE, phase="train")
        val_loss, val_acc = run_epoch(model, val_dl, criterion, None, DEVICE, phase="val")

        print(f"[Ep{epoch:02d}] Train acc={tr_acc:.3f}, loss={tr_loss:.3f} | Val acc={val_acc:.3f}, loss={val_loss:.3f}")

        scheduler.step(val_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            epochs_no_improve = 0
            torch.save(best_state, save_path)
            print(f"  -> New best val acc={best_val_acc:.3f} (model saved)")
        else:
            epochs_no_improve += 1
            print(f"  -> No improvement for {epochs_no_improve} epoch(s)")

        if epochs_no_improve >= ES_PATIENCE:
            print(f"[EarlyStopping] {ES_PATIENCE} epoch 동안 개선이 없어 중단합니다.")
            break

    # 8) Best 모델 로드 후 Test 평가
    if best_state is not None:
        model.load_state_dict(best_state)
        model.to(DEVICE)
        print(f"[INFO] Loaded best model from {save_path}")

    test_acc, cm = eval_with_confusion(model, test_dl, DEVICE)
    print(f"\n✅ Final Test Accuracy = {test_acc:.3f}")
    print("Confusion Matrix:")
    print(cm)

Writing train_seq_cnn_from_hands_mobilenet_v2.py


In [2]:
!python train_seq_cnn_from_hands_mobilenet_v2.py

[INFO] train_seq_cnn_from_hands_mobilenet_v2.py loaded
[INFO] Device = cuda
[SCAN] Person 1: 1712 frames
[SCAN] Person 10: 1760 frames
[SCAN] Person 2: 1760 frames
[SCAN] Person 3: 1760 frames
[SCAN] Person 4: 1760 frames
[SCAN] Person 5: 1760 frames
[SCAN] Person 6: 1760 frames
[SCAN] Person 7: 1760 frames
[SCAN] Person 8: 1760 frames
[SCAN] Person 9: 1760 frames
[META] Total sequence samples = 1097
[META] num_classes = 22, labels = ['0029' '0033' '0036' '0037' '0039' '0040' '0041' '0042' '0046' '0062'
 '0064' '0065' '0163' '0187' '0400' '0572' '0689' '0885' '1115' '1129'
 '1158' '1496']
[SPLIT] train=767, val=165, test=165
[INFO] class weights: [1.      1.      1.      1.      1.      1.      1.      1.      1.
 1.      1.      1.      1.      1.      1.09375 1.      1.      1.
 1.      1.      1.      1.     ]
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100% 9.83M/9.83M [00:0

2D Hands Image 기반 시퀀스 분류 (MobileNetV3 + TCN + Attention)

In [ ]:
%%writefile train_seq_cnn_from_hands_mobilenet_tcn.py
"""
2D Hands Image 기반 시퀀스 분류 (MobileNetV3 + TCN + Attention)

1) /Holistic_hands_frames/1~10 폴더의 PNG를 직접 읽어서 시퀀스 구성
   - 파일 예시: WORD1496_..._REAL01_U_s00.png ~ s15.png
   - base_id = '..._REAL01_U' 단위로 한 시퀀스로 그룹
2) 각 base_id 를 하나의 시퀀스(sample)로 사용 (길이 T=SEQ_LEN)
3) 라벨은 파일명 내 WORDxxxx 에서 추출 (xxxx 문자열)
4) 전체 시퀀스를 라벨 기준 stratified 0.7 / 0.15 / 0.15 split
5) MobileNetV3 Small 프레임 인코더 + TCN + Attention으로 분류 학습
   - Input: (B, T, 3, H, W)  (2D 이미지를 그대로 사용)
   - 프레임 인코더: MobileNetV3 Small (ImageNet pretrained) → 256차원
   - 시퀀스 인코더: TCN (hidden_channels=256, dilations=1,2,4)
   - Attention Pooling
   - Class weight 적용
   - Label smoothing
   - 프레임 레벨 Gaussian noise + Temporal dropout (train 시에만)
   - ReduceLROnPlateau 스케줄러 + EarlyStopping
"""

import os
import glob
import re
import random
from typing import List, Dict, Any, Tuple

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset  # Subset은 사용하지 않아도 무방

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

import torchvision.transforms as transforms
import torchvision.models as models

print("[INFO] train_seq_cnn_from_hands_mobilenet_tcn.py loaded")

# -------------------------------------------------------------
# 설정
# -------------------------------------------------------------
ROOT = "/content/drive/MyDrive/cv-medislr/data/preprocessed/holistic_frames/hands"

SEQ_LEN = 16       # 시퀀스 길이 (s00~s15 기준)
IMG_SIZE = 96      # 입력 이미지 크기 (HxW)
BATCH = 32         # Colab CPU가 부담되면 16으로 줄여도 됨
EPOCHS = 40
LR = 1e-3
WEIGHT_DECAY = 1e-4
ES_PATIENCE = 7    # Early stopping patience

# 프레임 레벨 데이터 증강
IMG_NOISE_STD = 0.01     # 0이면 비활성화
TEMP_DROP_PROB = 0.10    # 0이면 비활성화 (프레임 드롭)

RANDOM_SEED = 42

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Device = {DEVICE}")

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if DEVICE.type == "cuda":
    torch.cuda.manual_seed_all(RANDOM_SEED)

# -------------------------------------------------------------
# 1. 시퀀스 메타 수집 (2D PNG → 시퀀스 경로 리스트)
# -------------------------------------------------------------

# 예:  WORD1496_..._REAL01_U_s15.png  → base_id='WORD1496_..._REAL01_U', seg_idx=15
SEQ_GROUP_RE = re.compile(r"(.+)_s(\d+)\.png$")


def collect_sequences_from_png(root_dir: str) -> Tuple[List[List[str]], List[str]]:
    """
    각 person 폴더(1~10) 아래 PNG 파일을 읽어서
    base_id 단위 시퀀스를 구성하고, 라벨 문자열을 추출합니다.

    Returns
    -------
    sequences : List[List[str]]
        각 원소는 한 시퀀스를 구성하는 frame path 리스트 (길이 가변)
    labels_str : List[str]
        각 시퀀스에 대응되는 라벨 문자열 (예: "0029")
    """
    sequences: List[List[str]] = []
    labels_str: List[str] = []

    # 1,2,...,10 폴더만 사용 (seq_embeddings_hands 등은 제외)
    person_dirs = sorted(
        [
            d for d in glob.glob(os.path.join(root_dir, "*"))
            if os.path.isdir(d) and os.path.basename(d).isdigit()
        ]
    )

    for p_dir in person_dirs:
        person_id = os.path.basename(p_dir)
        png_files = sorted(glob.glob(os.path.join(p_dir, "*.png")))
        print(f"[SCAN] Person {person_id}: {len(png_files)} frames")

        # base_id 기준 그룹핑
        groups: Dict[str, Dict[int, str]] = {}

        for img_path in png_files:
            fname = os.path.basename(img_path)
            m = SEQ_GROUP_RE.match(fname)
            if not m:
                continue
            base_id, seg_str = m.groups()
            seg_idx = int(seg_str)

            if base_id not in groups:
                groups[base_id] = {}
            # 같은 seg_idx에 여러 파일이 있을 가능성은 거의 없지만, 있다면 가장 앞의 것만 사용
            if seg_idx not in groups[base_id]:
                groups[base_id][seg_idx] = img_path

        # base_id 하나가 하나의 시퀀스가 됨
        for base_id, seg_dict in groups.items():
            seg_indices = sorted(seg_dict.keys())
            frame_paths = [seg_dict[i] for i in seg_indices]

            if len(frame_paths) == 0:
                continue

            # 라벨 문자열 추출: base_id 내 WORDxxxx 사용
            label_txt = "0000"
            try:
                parts = base_id.split("_")
                word_token = next(
                    (p for p in parts if p.startswith("WORD") and p[4:].isdigit()),
                    None,
                )
                if word_token is not None:
                    label_txt = word_token.replace("WORD", "")
            except Exception:
                pass

            sequences.append(frame_paths)
            labels_str.append(label_txt)

    print(f"[META] Total sequence samples = {len(sequences)}")
    return sequences, labels_str


# -------------------------------------------------------------
# 2. Dataset 정의
# -------------------------------------------------------------
class HandsSeqDataset(Dataset):
    """
    sequences: List[List[str]]  (각 원소는 frame path 리스트)
    labels   : List[int]        (encoded label)
    indices  : Subset 인덱스
    """

    def __init__(
        self,
        sequences: List[List[str]],
        labels: List[int],
        indices: List[int],
        transform: transforms.Compose,
    ):
        self.sequences = sequences
        self.labels = labels
        self.indices = indices
        self.transform = transform

    def __len__(self) -> int:
        return len(self.indices)

    def _select_frames(self, frame_paths: List[str]) -> List[str]:
        """
        길이가 가변인 frame_paths를 SEQ_LEN 길이로 변환.
        - 길이가 SEQ_LEN보다 길면 균등 간격으로 샘플링
        - 짧으면 마지막 프레임을 반복하여 패딩
        """
        L = len(frame_paths)
        if L == 0:
            raise ValueError("Empty frame list encountered.")

        if L == SEQ_LEN:
            return frame_paths
        elif L > SEQ_LEN:
            idxs = np.linspace(0, L - 1, SEQ_LEN)
            idxs = np.round(idxs).astype(int)
            idxs = np.clip(idxs, 0, L - 1)
            return [frame_paths[i] for i in idxs]
        else:
            pad_num = SEQ_LEN - L
            return frame_paths + [frame_paths[-1]] * pad_num

    def __getitem__(self, idx: int):
        real_idx = self.indices[idx]
        frame_paths = self.sequences[real_idx]
        frame_paths = self._select_frames(frame_paths)

        imgs = []
        for path in frame_paths:
            img = Image.open(path).convert("RGB")
            img = self.transform(img)  # (3, H, W)
            imgs.append(img)
        # (T, 3, H, W)
        seq_tensor = torch.stack(imgs, dim=0)
        label = self.labels[real_idx]
        return seq_tensor, label


# -------------------------------------------------------------
# 3. MobileNetV3 프레임 인코더
# -------------------------------------------------------------
class FrameEncoderMobileNetV3(nn.Module):
    """
    ImageNet pretrained MobileNetV3 Small을 프레임 인코더로 사용.
    입력: (B, 3, H, W) → 출력: (B, out_dim)
    """

    def __init__(self, out_dim: int = 256, pretrained: bool = True):
        super().__init__()
        weights = models.MobileNet_V3_Small_Weights.DEFAULT if pretrained else None
        backbone = models.mobilenet_v3_small(weights=weights)

        # feature extractor (classifier 제거)
        self.features = backbone.features
        self.gap = nn.AdaptiveAvgPool2d((1, 1))

        in_feat = backbone.classifier[0].in_features
        self.proj = nn.Linear(in_feat, out_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, 3, H, W)
        f = self.features(x)              # (B, C, H', W')
        f = self.gap(f).flatten(1)        # (B, C)
        f = self.proj(f)                  # (B, out_dim)
        return f


# -------------------------------------------------------------
# 4. TCN 정의
# -------------------------------------------------------------
class TemporalBlock(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int = 3,
        dilation: int = 1,
        dropout: float = 0.2,
    ):
        super().__init__()
        padding = (kernel_size - 1) * dilation // 2  # 길이 유지용

        self.conv1 = nn.Conv1d(
            in_channels,
            out_channels,
            kernel_size,
            padding=padding,
            dilation=dilation,
        )
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv1d(
            out_channels,
            out_channels,
            kernel_size,
            padding=padding,
            dilation=dilation,
        )
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU(inplace=True)

        self.dropout = nn.Dropout(dropout)

        self.downsample = (
            nn.Conv1d(in_channels, out_channels, kernel_size=1)
            if in_channels != out_channels
            else None
        )
        self.final_relu = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, C, T)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.dropout(out)

        res = x if self.downsample is None else self.downsample(x)
        out = out + res
        out = self.final_relu(out)
        return out


class TemporalConvNet(nn.Module):
    def __init__(
        self,
        input_channels: int,
        hidden_channels: int = 256,
        num_layers: int = 3,
        kernel_size: int = 3,
        dropout: float = 0.2,
    ):
        super().__init__()
        layers = []
        in_ch = input_channels
        for i in range(num_layers):
            dilation = 2**i  # 1, 2, 4, ...
            layers.append(
                TemporalBlock(
                    in_channels=in_ch,
                    out_channels=hidden_channels,
                    kernel_size=kernel_size,
                    dilation=dilation,
                    dropout=dropout,
                )
            )
            in_ch = hidden_channels
        self.network = nn.Sequential(*layers)
        self.out_channels = hidden_channels

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, C, T)
        return self.network(x)


# -------------------------------------------------------------
# 5. MobileNet + TCN + Attention 모델
# -------------------------------------------------------------
class SeqCNN_MobileNet_TCN_Attn(nn.Module):
    """
    2D 이미지 → MobileNetV3 Small → TCN → Attention → 분류
    CNN 쪽은 GRU 버전과 동일하게 두고, 시퀀스 인코더만 TCN으로 교체.
    """

    def __init__(
        self,
        num_classes: int,
        frame_out_dim: int = 256,
        tcn_hidden: int = 256,
        tcn_layers: int = 3,
        dropout: float = 0.2,
        pretrained_backbone: bool = True,
    ):
        super().__init__()
        self.encoder = FrameEncoderMobileNetV3(
            out_dim=frame_out_dim,
            pretrained=pretrained_backbone,
        )

        # TCN: 입력 채널 = frame_out_dim, 출력 채널 = tcn_hidden
        self.tcn = TemporalConvNet(
            input_channels=frame_out_dim,
            hidden_channels=tcn_hidden,
            num_layers=tcn_layers,
            kernel_size=3,
            dropout=dropout,
        )
        out_dim = self.tcn.out_channels

        self.attn_fc = nn.Linear(out_dim, 1)
        self.head = nn.Sequential(
            nn.LayerNorm(out_dim),
            nn.Dropout(0.1),
            nn.Linear(out_dim, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, T, 3, H, W)
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)

        # 1) 프레임별 CNN 인코딩
        feat = self.encoder(x)               # (B*T, F)
        feat = feat.view(B, T, -1)           # (B, T, F)

        # 2) TCN: (B, F, T) → (B, H, T)
        feat_t = feat.permute(0, 2, 1)       # (B, F, T)
        out = self.tcn(feat_t)               # (B, H, T)
        out = torch.nan_to_num(out, nan=0.0, posinf=0.0, neginf=0.0)

        # 3) Attention over time
        out_seq = out.permute(0, 2, 1)       # (B, T, H)
        w = self.attn_fc(out_seq)            # (B, T, 1)
        w = torch.softmax(w, dim=1)          # (B, T, 1)
        w = torch.nan_to_num(w, nan=0.0, posinf=0.0, neginf=0.0)

        feat_seq = (w * out_seq).sum(dim=1)  # (B, H)
        logits = self.head(feat_seq)         # (B, num_classes)
        return logits


# -------------------------------------------------------------
# 6. Train / Eval loop  (GRU 버전과 동일 패턴)
# -------------------------------------------------------------
def run_epoch(
    model: nn.Module,
    loader: DataLoader,
    criterion,
    optimizer=None,
    device="cpu",
    phase: str = "train",
):
    train_flag = optimizer is not None
    model.train(train_flag)

    total, correct, loss_sum = 0, 0, 0.0

    for xb, yb in loader:
        # xb: (B, T, 3, H, W)
        xb = xb.to(device)
        yb = yb.to(device)

        # ----- frame-level augmentation (train 시에만)-----
        if train_flag:
            if IMG_NOISE_STD > 0:
                noise = torch.randn_like(xb) * IMG_NOISE_STD
                xb = xb + noise

            if TEMP_DROP_PROB > 0:
                # 일부 프레임 전체를 drop (0으로)
                B_, T_, C_, H_, W_ = xb.shape
                mask = (
                    torch.rand(B_, T_, 1, 1, 1, device=device)
                    > TEMP_DROP_PROB
                ).float()
                xb = xb * mask

        if train_flag:
            optimizer.zero_grad(set_to_none=True)

        logits = model(xb)
        loss = criterion(logits, yb)

        if train_flag:
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 3.0)
            optimizer.step()

        with torch.no_grad():
            pred = logits.argmax(dim=1)
            total += yb.size(0)
            correct += (pred == yb).sum().item()
            loss_sum += loss.item() * yb.size(0)

    avg_loss = loss_sum / max(1, total)
    acc = correct / max(1, total)
    return avg_loss, acc


def eval_with_confusion(model, loader, device="cpu"):
    model.eval()
    all_y, all_p = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)
            logits = model(xb)
            pred = logits.argmax(dim=1)
            all_y += yb.cpu().tolist()
            all_p += pred.cpu().tolist()
    cm = confusion_matrix(all_y, all_p)
    acc = (cm.diagonal().sum() / cm.sum()) if cm.sum() > 0 else 0.0
    return acc, cm


# -------------------------------------------------------------
# 7. main
# -------------------------------------------------------------
if __name__ == "__main__":
    # 1) 시퀀스 메타 수집
    seq_paths, labels_str = collect_sequences_from_png(ROOT)

    if len(seq_paths) == 0:
        raise ValueError("수집된 시퀀스가 없습니다. 폴더 구조와 파일명을 확인하세요.")

    # 라벨 인코딩 (문자열 → 정수)
    encoder = LabelEncoder()
    encoder.fit(labels_str)
    labels = encoder.transform(labels_str).astype(np.int64)
    num_classes = len(encoder.classes_)
    print(f"[META] num_classes = {num_classes}, labels = {encoder.classes_}")

    # 2) Stratified random split
    y_all = labels
    indices = np.arange(len(seq_paths))

    train_idx, temp_idx = train_test_split(
        indices,
        test_size=0.3,
        stratify=y_all,
        random_state=RANDOM_SEED,
    )
    val_idx, test_idx = train_test_split(
        temp_idx,
        test_size=0.5,
        stratify=y_all[temp_idx],
        random_state=RANDOM_SEED,
    )

    print(f"[SPLIT] train={len(train_idx)}, val={len(val_idx)}, test={len(test_idx)}")

    # 3) class weight 계산
    class_counts = np.bincount(y_all[train_idx], minlength=num_classes)
    max_count = class_counts.max()
    class_weights = (max_count / class_counts).astype(np.float32)
    print("[INFO] class weights:", class_weights)

    # 4) Transform 정의 (ImageNet 정규화 사용)
    imagenet_mean = [0.485, 0.456, 0.406]
    imagenet_std = [0.229, 0.224, 0.225]

    transform_train = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])

    transform_eval = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])

    # 5) Dataset / DataLoader
    train_set = HandsSeqDataset(seq_paths, y_all.tolist(), train_idx.tolist(), transform_train)
    val_set   = HandsSeqDataset(seq_paths, y_all.tolist(), val_idx.tolist(), transform_eval)
    test_set  = HandsSeqDataset(seq_paths, y_all.tolist(), test_idx.tolist(), transform_eval)

    pin = DEVICE.type == "cuda"
    train_dl = DataLoader(train_set, batch_size=BATCH, shuffle=True,  num_workers=2, pin_memory=pin)
    val_dl   = DataLoader(val_set,   batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=pin)
    test_dl  = DataLoader(test_set,  batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=pin)

    # 6) 모델 정의 (MobileNet backbone 동일, GRU 대신 TCN)
    model = SeqCNN_MobileNet_TCN_Attn(
        num_classes=num_classes,
        frame_out_dim=256,
        tcn_hidden=256,
        tcn_layers=3,
        dropout=0.2,
        pretrained_backbone=True,
    ).to(DEVICE)

    # 필요하면 backbone freeze (GRU 모델과 동일하게 맞추고 싶다면 둘 다 같은 설정 사용)
    # for p in model.encoder.features.parameters():
    #     p.requires_grad = False

    weight_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)
    criterion = nn.CrossEntropyLoss(weight=weight_tensor, label_smoothing=0.05)

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LR,
        weight_decay=WEIGHT_DECAY,
    )

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode="max",
        factor=0.5,
        patience=3,
        min_lr=1e-5,
    )

    best_val_acc = 0.0
    best_state = None
    epochs_no_improve = 0

    save_path = os.path.join(ROOT, "seq_cnn_mobilenet_tcn_attn_hands_best.pt")

    # 7) 학습 루프
    for epoch in range(1, EPOCHS + 1):
        tr_loss, tr_acc = run_epoch(model, train_dl, criterion, optimizer, DEVICE, phase="train")
        val_loss, val_acc = run_epoch(model, val_dl, criterion, None, DEVICE, phase="val")

        print(f"[Ep{epoch:02d}] Train acc={tr_acc:.3f}, loss={tr_loss:.3f} | "
              f"Val acc={val_acc:.3f}, loss={val_loss:.3f}")

        scheduler.step(val_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            epochs_no_improve = 0
            torch.save(best_state, save_path)
            print(f"  -> New best val acc={best_val_acc:.3f} (model saved)")
        else:
            epochs_no_improve += 1
            print(f"  -> No improvement for {epochs_no_improve} epoch(s)")

        if epochs_no_improve >= ES_PATIENCE:
            print(f"[EarlyStopping] {ES_PATIENCE} epoch 동안 개선이 없어 중단합니다.")
            break

    # 8) Best 모델 로드 후 Test 평가
    if best_state is not None:
        model.load_state_dict(best_state)
        model.to(DEVICE)
        print(f"[INFO] Loaded best model from {save_path}")

    test_acc, cm = eval_with_confusion(model, test_dl, DEVICE)
    print(f"\n✅ Final Test Accuracy = {test_acc:.3f}")
    print("Confusion Matrix:")
    print(cm)


Overwriting train_seq_cnn_from_hands_mobilenet_tcn.py


In [ ]:
!python train_seq_cnn_from_hands_mobilenet_tcn.py

[INFO] train_seq_cnn_from_hands_mobilenet_tcn.py loaded
[INFO] Device = cpu
[SCAN] Person 1: 1712 frames
[SCAN] Person 10: 1760 frames
[SCAN] Person 2: 1760 frames
[SCAN] Person 3: 1760 frames
[SCAN] Person 4: 1760 frames
[SCAN] Person 5: 1760 frames
[SCAN] Person 6: 1760 frames
[SCAN] Person 7: 1760 frames
[SCAN] Person 8: 1760 frames
[SCAN] Person 9: 1760 frames
[META] Total sequence samples = 1097
[META] num_classes = 22, labels = ['0029' '0033' '0036' '0037' '0039' '0040' '0041' '0042' '0046' '0062'
 '0064' '0065' '0163' '0187' '0400' '0572' '0689' '0885' '1115' '1129'
 '1158' '1496']
[SPLIT] train=767, val=165, test=165
[INFO] class weights: [1.      1.      1.      1.      1.      1.      1.      1.      1.
 1.      1.      1.      1.      1.      1.09375 1.      1.      1.
 1.      1.      1.      1.     ]
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100% 9.83M/9.83M [00:0

전처리 파일 저장

In [1]:
%%writefile preprocess_testset_to_pt_only_tcn.py
"""
TEST SET 전처리 (TCN 실험용)
- 모델 생성 없음
- 학습 없음
- PNG → 동일 transform → tensor → .pt 저장
- GRU 버전과 split / 전처리 100% 동일
- 저장 경로: data/samples/2D Sequence/tcn
"""

import os
import glob
import re
import random
from typing import List, Dict

import numpy as np
import pandas as pd
from PIL import Image

import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchvision.transforms as transforms

# -------------------------------------------------------------
# 경로 설정 (✅ 최종 확정)
# -------------------------------------------------------------
# 전처리된 hand frame PNG들이 있는 위치
ROOT = "/content/drive/MyDrive/cv-medislr/data/preprocessed/holistic_frames/hands"

# 샘플 저장 루트 (요구된 경로)
SAVE_ROOT = "/content/drive/MyDrive/cv-medislr/data/samples/2D Sequence"

# TCN 실험용 tensor 저장 경로
SAVE_TENSOR_DIR = os.path.join(SAVE_ROOT, "tcn")

# metadata 저장 경로
SAVE_META_DIR = os.path.join(SAVE_ROOT, "metadata")

os.makedirs(SAVE_TENSOR_DIR, exist_ok=True)
os.makedirs(SAVE_META_DIR, exist_ok=True)

# -------------------------------------------------------------
# 공통 설정
# -------------------------------------------------------------
SEQ_LEN = 16
IMG_SIZE = 96
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# -------------------------------------------------------------
# transform (eval 기준, 학습 코드와 동일)
# -------------------------------------------------------------
transform_eval = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225],
    ),
])

# -------------------------------------------------------------
# 시퀀스 수집
# -------------------------------------------------------------
SEQ_GROUP_RE = re.compile(r"(.+)_s(\d+)\.png$")


def collect_sequences(root_dir: str):
    sequences, labels_str, base_ids = [], [], []

    person_dirs = sorted(
        d for d in glob.glob(os.path.join(root_dir, "*"))
        if os.path.isdir(d) and os.path.basename(d).isdigit()
    )

    for p_dir in person_dirs:
        png_files = sorted(glob.glob(os.path.join(p_dir, "*.png")))
        groups: Dict[str, Dict[int, str]] = {}

        for img_path in png_files:
            fname = os.path.basename(img_path)
            m = SEQ_GROUP_RE.match(fname)
            if not m:
                continue

            base_id, seg = m.groups()
            groups.setdefault(base_id, {})[int(seg)] = img_path

        for base_id, seg_dict in groups.items():
            frames = [seg_dict[i] for i in sorted(seg_dict)]
            if not frames:
                continue

            # WORDxxxx 라벨 추출
            label_txt = "0000"
            for part in base_id.split("_"):
                if part.startswith("WORD") and part[4:].isdigit():
                    label_txt = part.replace("WORD", "")
                    break

            sequences.append(frames)
            labels_str.append(label_txt)
            base_ids.append(base_id)

    print(f"[META] Total sequences = {len(sequences)}")
    return sequences, labels_str, base_ids


def select_frames(frames: List[str]) -> List[str]:
    """
    학습 코드와 완전히 동일한 frame selection 로직
    """
    L = len(frames)
    if L >= SEQ_LEN:
        idxs = np.linspace(0, L - 1, SEQ_LEN).round().astype(int)
        return [frames[i] for i in idxs]
    else:
        return frames + [frames[-1]] * (SEQ_LEN - L)


# -------------------------------------------------------------
# main
# -------------------------------------------------------------
if __name__ == "__main__":
    print("[INFO] Collecting sequences (TCN)...")
    sequences, labels_str, base_ids = collect_sequences(ROOT)

    # label encoding (split 재현용)
    le = LabelEncoder()
    labels = le.fit_transform(labels_str)

    indices = np.arange(len(sequences))

    # 🔑 이전 TCN 실험과 동일한 split 재현
    _, temp_idx = train_test_split(
        indices,
        test_size=0.3,
        stratify=labels,
        random_state=RANDOM_SEED,
    )
    _, test_idx = train_test_split(
        temp_idx,
        test_size=0.5,
        stratify=labels[temp_idx],
        random_state=RANDOM_SEED,
    )

    print(f"[SPLIT] TCN test samples = {len(test_idx)}")

    # ---------------------------------------------------------
    # TEST SET 전처리 → .pt 저장
    # ---------------------------------------------------------
    meta_rows = []

    for i, idx in enumerate(test_idx):
        frames = select_frames(sequences[idx])

        imgs = []
        for p in frames:
            img = Image.open(p).convert("RGB")
            imgs.append(transform_eval(img))

        tensor = torch.stack(imgs)  # (T, 3, H, W)

        save_name = f"seq_{i:05d}.pt"
        save_path = os.path.join(SAVE_TENSOR_DIR, save_name)

        torch.save({
            "x": tensor,
            "y": int(labels[idx]),
            "base_id": base_ids[idx],
            "split": "test",
            "model": "TCN",
        }, save_path)

        meta_rows.append({
            "seq_file": save_name,
            "base_id": base_ids[idx],
            "label": int(labels[idx]),
            "split": "test",
            "model": "TCN",
        })

    # metadata CSV 저장
    meta_csv = os.path.join(SAVE_META_DIR, "tsn_hands_test_samples_tcn.csv")
    pd.DataFrame(meta_rows).to_csv(meta_csv, index=False)

    print(f"[DONE] Saved {len(test_idx)} TCN test tensors.")
    print(f"[DONE] Metadata saved to {meta_csv}")


Writing preprocess_testset_to_pt_only_tcn.py


In [2]:
! python preprocess_testset_to_pt_only_tcn.py

[INFO] Collecting sequences (TCN)...
[META] Total sequences = 1097
[SPLIT] TCN test samples = 165
[DONE] Saved 165 TCN test tensors.
[DONE] Metadata saved to /content/drive/MyDrive/cv-medislr/data/samples/2D Sequence/metadata/tsn_hands_test_samples_tcn.csv


In [3]:
%%writefile preprocess_testset_to_pt_only_gru.py
"""
TEST SET 전처리 (GRU 실험용)
- 모델 생성 없음
- 학습 없음
- PNG → 동일 transform → tensor → .pt 저장
- TCN 버전과 split / 전처리 100% 동일
"""

import os, glob, re, random
from typing import List, Dict

import numpy as np
import pandas as pd
from PIL import Image
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchvision.transforms as transforms

# -------------------------------------------------------------
# 경로 설정 (요청한 형식 그대로)
# -------------------------------------------------------------
ROOT = "/content/drive/MyDrive/cv-medislr/data/preprocessed/holistic_frames/hands"

SAVE_ROOT = "/content/drive/MyDrive/cv-medislr/data/samples/2D Sequence"
SAVE_TENSOR_DIR = f"{SAVE_ROOT}/gru"
SAVE_META_DIR   = f"{SAVE_ROOT}/metadata"

os.makedirs(SAVE_TENSOR_DIR, exist_ok=True)
os.makedirs(SAVE_META_DIR, exist_ok=True)

# -------------------------------------------------------------
# 공통 설정
# -------------------------------------------------------------
SEQ_LEN = 16
IMG_SIZE = 96
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# -------------------------------------------------------------
# transform (eval 기준, 학습 코드와 동일)
# -------------------------------------------------------------
transform_eval = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225],
    ),
])

# -------------------------------------------------------------
# 시퀀스 수집
# -------------------------------------------------------------
SEQ_GROUP_RE = re.compile(r"(.+)_s(\d+)\.png$")

def collect_sequences(root_dir):
    sequences, labels_str, base_ids = [], [], []

    person_dirs = sorted(
        d for d in glob.glob(os.path.join(root_dir, "*"))
        if os.path.isdir(d) and os.path.basename(d).isdigit()
    )

    for p_dir in person_dirs:
        pngs = sorted(glob.glob(os.path.join(p_dir, "*.png")))
        groups: Dict[str, Dict[int, str]] = {}

        for p in pngs:
            m = SEQ_GROUP_RE.match(os.path.basename(p))
            if not m:
                continue
            base_id, seg = m.groups()
            groups.setdefault(base_id, {})[int(seg)] = p

        for base_id, seg_dict in groups.items():
            frames = [seg_dict[i] for i in sorted(seg_dict)]
            if not frames:
                continue

            # WORDxxxx 라벨 추출
            label_txt = "0000"
            for part in base_id.split("_"):
                if part.startswith("WORD") and part[4:].isdigit():
                    label_txt = part.replace("WORD", "")
                    break

            sequences.append(frames)
            labels_str.append(label_txt)
            base_ids.append(base_id)

    print(f"[META] Total sequences = {len(sequences)}")
    return sequences, labels_str, base_ids


def select_frames(frames: List[str]) -> List[str]:
    """
    학습 코드와 완전히 동일한 frame selection
    """
    L = len(frames)
    if L >= SEQ_LEN:
        idxs = np.linspace(0, L - 1, SEQ_LEN).round().astype(int)
        return [frames[i] for i in idxs]
    return frames + [frames[-1]] * (SEQ_LEN - L)


# -------------------------------------------------------------
# main
# -------------------------------------------------------------
if __name__ == "__main__":
    print("[INFO] Collecting sequences (GRU)...")
    sequences, labels_str, base_ids = collect_sequences(ROOT)

    # label encoding (split 재현용)
    le = LabelEncoder()
    labels = le.fit_transform(labels_str)

    indices = np.arange(len(sequences))

    # 🔑 TCN / GRU 동일한 test split 재현
    _, temp_idx = train_test_split(
        indices,
        test_size=0.3,
        stratify=labels,
        random_state=RANDOM_SEED,
    )
    _, test_idx = train_test_split(
        temp_idx,
        test_size=0.5,
        stratify=labels[temp_idx],
        random_state=RANDOM_SEED,
    )

    print(f"[SPLIT] GRU test samples = {len(test_idx)}")

    meta_rows = []

    for i, idx in enumerate(test_idx):
        frames = select_frames(sequences[idx])

        imgs = []
        for p in frames:
            img = Image.open(p).convert("RGB")
            imgs.append(transform_eval(img))

        tensor = torch.stack(imgs)  # (T, 3, H, W)

        fname = f"seq_{i:05d}.pt"
        torch.save(
            {
                "x": tensor,
                "y": int(labels[idx]),
                "base_id": base_ids[idx],
                "split": "test",
                "model": "GRU",
            },
            f"{SAVE_TENSOR_DIR}/{fname}",
        )

        meta_rows.append({
            "seq_file": fname,
            "base_id": base_ids[idx],
            "label": int(labels[idx]),
            "split": "test",
            "model": "GRU",
        })

    meta_csv = f"{SAVE_META_DIR}/tsn_hands_test_samples_gru.csv"
    pd.DataFrame(meta_rows).to_csv(meta_csv, index=False)

    print(f"[DONE] Saved {len(test_idx)} GRU test tensors.")
    print(f"[DONE] Metadata saved to {meta_csv}")


Writing preprocess_testset_to_pt_only_gru.py


In [4]:
!python preprocess_testset_to_pt_only_gru.py

[INFO] Collecting sequences (GRU)...
[META] Total sequences = 1097
[SPLIT] GRU test samples = 165
[DONE] Saved 165 GRU test tensors.
[DONE] Metadata saved to /content/drive/MyDrive/cv-medislr/data/samples/2D Sequence/metadata/tsn_hands_test_samples_gru.csv


랜덤 20개 추출

In [8]:
%%writefile make_demo_samples_from_test.py
"""
TEST pt 중에서 랜덤 20개를 추출해 demo/sample 폴더 생성
- 전처리 없음
- split 변경 없음
- test 정의 유지
- seed 고정 (재현 가능)
"""

import os
import random
import shutil
import pandas as pd

# -------------------------------------------------------------
# 설정
# -------------------------------------------------------------
RANDOM_SEED = 42
NUM_SAMPLES = 20
random.seed(RANDOM_SEED)

BASE_ROOT = "/content/drive/MyDrive/cv-medislr/data/samples/2D Sequence"

GRU_TEST_DIR = f"{BASE_ROOT}/gru"
TCN_TEST_DIR = f"{BASE_ROOT}/tcn"

DEMO_ROOT = f"{BASE_ROOT}/demo"
GRU_DEMO_DIR = f"{DEMO_ROOT}/gru_2dsequence"
TCN_DEMO_DIR = f"{DEMO_ROOT}/tcn_2dsequence"
META_DIR = f"{DEMO_ROOT}/metadata"

os.makedirs(GRU_DEMO_DIR, exist_ok=True)
os.makedirs(TCN_DEMO_DIR, exist_ok=True)
os.makedirs(META_DIR, exist_ok=True)

# -------------------------------------------------------------
# 파일 목록
# -------------------------------------------------------------
gru_files = sorted(f for f in os.listdir(GRU_TEST_DIR) if f.endswith(".pt"))
tcn_files = sorted(f for f in os.listdir(TCN_TEST_DIR) if f.endswith(".pt"))

assert len(gru_files) >= NUM_SAMPLES, "GRU test pt 개수 부족"
assert len(tcn_files) >= NUM_SAMPLES, "TCN test pt 개수 부족"

# -------------------------------------------------------------
# 랜덤 샘플링
# -------------------------------------------------------------
gru_selected = random.sample(gru_files, NUM_SAMPLES)
tcn_selected = random.sample(tcn_files, NUM_SAMPLES)

rows = []

# -------------------------------------------------------------
# 파일 복사
# -------------------------------------------------------------
for f in gru_selected:
    shutil.copy(
        os.path.join(GRU_TEST_DIR, f),
        os.path.join(GRU_DEMO_DIR, f),
    )
    rows.append({
        "seq_file": f,
        "model": "GRU",
        "source": "test",
    })

for f in tcn_selected:
    shutil.copy(
        os.path.join(TCN_TEST_DIR, f),
        os.path.join(TCN_DEMO_DIR, f),
    )
    rows.append({
        "seq_file": f,
        "model": "TCN",
        "source": "test",
    })

# -------------------------------------------------------------
# metadata 저장
# -------------------------------------------------------------
meta_path = f"{META_DIR}/demo_samples_20.csv"
pd.DataFrame(rows).to_csv(meta_path, index=False)

print(f"[DONE] GRU demo samples: {len(gru_selected)}")
print(f"[DONE] TCN demo samples: {len(tcn_selected)}")
print(f"[DONE] Metadata saved to {meta_path}")


Overwriting make_demo_samples_from_test.py


In [9]:
!python make_demo_samples_from_test.py

[DONE] GRU demo samples: 20
[DONE] TCN demo samples: 20
[DONE] Metadata saved to /content/drive/MyDrive/cv-medislr/data/samples/2D Sequence/demo/metadata/demo_samples_20.csv
